In [1]:
import numpy as np
import os
import time
import argparse
import random
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, BatchNormalization, Activation

from tensorflow.keras.optimizers import Adam
from qkeras import QActivation, QDense, QConv1D, QConv2D, quantized_bits
from qkeras.autoqkeras.utils import print_qmodel_summary

from sklearn.metrics import accuracy_score
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#from node_edge_projection import NodeEdgeProjection

sys.path.append("..")
import util.data
import util.plots
import util.util
from util.terminal_colors import tcols

os.environ['CUDA_VISIBLE_DEVICES'] = "0"


In [2]:
# replace with your own model_dir
arch="QMLP"
#model_dir = "model_QMLP_nconst8_nbits8_Seed175_Kfold_20230923-023935"
#model_dir = "model_QMLP_nconst16_nbits8_Seed175_Kfold_20230923-030231" 
#model_dir = "model_QMLP_nconst32_nbits8_Seed175_Kfold_20230923-032558"
#model_dir = "model_QMLP_nconst32_nbits8_Seed175_Kfold_20231023-131839_6Kparameters"
model_dir = "model_QMLP_nconst32_nbits8_Seed175_Kfold_20231023-190324_28Kparamters"


#arch="QGCN"
#model_dir = "model_QGCN_nconst8_nbits8_Seed175_Kfold_20230923-085954"
#model_dir = "model_QGCN_nconst16_nbits8_Seed175_Kfold_20230923-094948"
#model_dir = "model_QGCN_nconst32_nbits8_Seed175_Kfold_20230923-174141"


# Number of constituents
nmax = 32
nconstit = nmax
nfeat = 3

# Quantized bits
nbits = 8
integ = 0

In [3]:

fpath = f'./data_kfold/jets_{nmax}constituents_ptetaphi_nonorm'
fname_test = f'jet_images_c{nmax}_minpt2.0_ptetaphi_nonorm'

data = util.data.Data(fpath=fpath, fname=fname_test, only_test=True)

X_test = data.test_data
Y_test = data.test_target

interquantile_range_32 = [120, 0.27, 0.27]
interquantile_range_16 = [166, 0.24, 0.24]
interquantile_range_8  = [219, 0.20, 0.20]
    
if nmax == 8:
    X_test      = X_test      / interquantile_range_8
elif nmax == 16:
    X_test      = X_test      / interquantile_range_16
elif nmax == 32:
    X_test      = X_test      / interquantile_range_32

# Flatten data for MLP input
if (arch=="QMLP"):
    NINPUT=nconstit*nfeat
    X_test  = X_test.reshape(-1, NINPUT)

        
        
# The dataset is N_jets x N_constituents x N_features
njet = X_test.shape[0]

ntargets = Y_test.shape[1]
    
print("#jets = ", njet)
print("#constituents = ", nconstit)
print("#targets = ", ntargets)
print("#features = ", nfeat)
    
print(X_test.shape, Y_test.shape)


----------------
Data loading complete:
File name: jet_images_c32_minpt2.0_ptetaphi_nonorm
Training data size: 0
Test data size: 252,340
Number of constituents: 32
Number of features: 3
----------------

#jets =  252340
#constituents =  32
#targets =  5
#features =  3
(252340, 96) (252340, 5)


In [4]:

kfolds = 5
accuracy_keras = []
kfold_metrics = {
    "fprs": [],
    "aucs": [],
    "fats": [],
    "accs": [],
    "loss": []
}

with tf.device('CPU: 0'):   # disable M1 metal GPU 
  for i in range (kfolds):
    
    val_kfold = i
    #train_kfolds = [kfold for kfold in range(kfolds) if kfold != test_kfold]

    mname = "model_{}_nconst_{}_nbits_{}_kfold_{}".format(
            arch,
            nmax, 
            nbits,
            val_kfold,
    )
        
    model = tf.keras.models.load_model(
        "{}/{}.h5".format(model_dir, mname),compile=False,
        custom_objects={
            "QDense": QDense,
            "QActivation": QActivation,
            "QConv1D": QConv1D,
            "QConv2D": QConv2D,
            "quantized_bits": quantized_bits,
            #"GarNet": GarNet,
#            "NodeEdgeProjection": NodeEdgeProjection,
#            "PruneLowMagnitude": pruning_wrapper.PruneLowMagnitude,
        },
    )
    
    
    # Define the optimizer ( minimization algorithm )
    optim = Adam(learning_rate=0.0002)

    # compile the model
    model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.summary()

    
    y_keras = model.predict(X_test)
    
    accuracy_keras.append(float(
        accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))
    ))
    
    acc = keras.metrics.CategoricalAccuracy()
    acc.update_state(Y_test, y_keras)
    
    categorical_crossent = keras.losses.CategoricalCrossentropy()
    ce_loss = categorical_crossent(Y_test, y_keras).numpy()
    
    plots_dir = model_dir
    roc_metrics = util.plots.roc_curves(plots_dir, y_keras, Y_test, val_kfold)
    util.plots.dnn_output(plots_dir, y_keras)
    
    model_kfold_metrics = {}
    model_kfold_metrics.update({"fprs": roc_metrics[0]})
    model_kfold_metrics.update({"aucs": roc_metrics[2]})
    model_kfold_metrics.update({"fats": roc_metrics[3]})
    model_kfold_metrics.update({"accs": acc.result().numpy()})
    model_kfold_metrics.update({"loss": ce_loss})

    for key in model_kfold_metrics.keys():
        kfold_metrics[key].append(model_kfold_metrics[key])
        
    
print("inference done")    

Metal device set to: Apple M1


2023-10-23 20:57:26.891049: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-23 20:57:26.891157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp (InputLayer)            [(None, 96)]              0         
                                                                 
 batchnorm (BatchNormalizati  (None, 96)               384       
 on)                                                             
                                                                 
 hidden1 (QDense)            (None, 96)                9312      
                                                                 
 activation1 (QActivation)   (None, 96)                0         
                                                                 
 hidden2 (QDense)            (None, 96)                9312      
                                                                 
 activation2 (QActivation)   (None, 96)                0         
                                                             

2023-10-23 20:57:27.682807: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ROC curves plot saved to model_QMLP_nconst32_nbits8_Seed175_Kfold_20231023-190324_28Kparamters.
DNN output histograms plot saved to model_QMLP_nconst32_nbits8_Seed175_Kfold_20231023-190324_28Kparamters.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp (InputLayer)            [(None, 96)]              0         
                                                                 
 batchnorm (BatchNormalizati  (None, 96)               384       
 on)                                                             
                                                                 
 hidden1 (QDense)            (None, 96)                9312      
                                                                 
 activation1 (QActivation)   (None, 96)                0         
                                                                 
 hidden2 (QDense)            (None, 96)                9

In [5]:
print(kfold_metrics)

accuracy_average = np.mean(np.array(accuracy_keras))
accuracy_errs = np.std(np.array(accuracy_keras))
print(accuracy_average)
print(accuracy_errs)
print(accuracy_keras)



{'fprs': [[array([3.56661647e-04, 4.75548863e-04, 6.53879686e-04, 8.32210510e-04,
       1.09475311e-03, 1.34738844e-03, 1.64460648e-03, 1.97154633e-03,
       2.33811524e-03, 2.82852501e-03, 3.23472299e-03, 3.75485456e-03,
       4.22544979e-03, 4.81493223e-03, 5.47376555e-03, 6.09296980e-03,
       6.75675676e-03, 7.53943093e-03, 8.30724419e-03, 9.08991836e-03,
       9.82800983e-03, 1.06750812e-02, 1.16410399e-02, 1.25921376e-02,
       1.36324007e-02, 1.46478957e-02, 1.57178806e-02, 1.69067528e-02,
       1.80956250e-02, 1.93588016e-02, 2.06814219e-02, 2.19346913e-02,
       2.32077752e-02, 2.45006737e-02, 2.59719030e-02, 2.71904969e-02,
       2.87310771e-02, 3.04995245e-02, 3.22432036e-02, 3.39125783e-02,
       3.57454228e-02, 3.74891020e-02, 3.94210193e-02, 4.15164064e-02,
       4.34334628e-02, 4.55288500e-02, 4.77233098e-02, 5.00961005e-02,
       5.26472220e-02, 5.50497345e-02, 5.76850678e-02, 6.03996592e-02,
       6.32479987e-02, 6.60913846e-02, 6.92072204e-02, 7.27342078e

In [6]:
tprs_baseline = roc_metrics[1]


avg_metrics = {}
for key, value in kfold_metrics.items():
    avg_metrics.update(
        {f"{key}": np.mean(value, axis=0), f"{key}_errs": np.std(value, axis=0)}
    )
variance_fats = np.var(kfold_metrics['fats'], axis=0)

roc_uncert_plot_data = avg_metrics.copy()
del roc_uncert_plot_data['accs']
del roc_uncert_plot_data['accs_errs']
del roc_uncert_plot_data['loss']
del roc_uncert_plot_data['loss_errs']


In [7]:

roc_uncert_plot_data.update({'tpr': tprs_baseline, 'outdir': model_dir})
util.plots.roc_curves_uncert(**roc_uncert_plot_data)

inverse_fats = 1 / np.mean(avg_metrics['fats'])
inverse_fats_error = inverse_fats**2 * np.sqrt(np.sum(variance_fats))

print(f"Accuracy: {avg_metrics['accs']:.3f} \u00B1 {avg_metrics['accs_errs']:.3f}")
print(f"Avg loss: {avg_metrics['loss']:.3f} \u00B1 {avg_metrics['loss_errs']:.3f}")
print(f"Average 1/<FPR>: {inverse_fats:.3f} \u00B1 {inverse_fats_error:.3f}")


    
    

ROC curves plot saved to model_QMLP_nconst32_nbits8_Seed175_Kfold_20231023-190324_28Kparamters.
Accuracy: 0.617 ± 0.009
Avg loss: 0.969 ± 0.018
Average 1/<FPR>: 4.890 ± 0.680
